In [ ]:
from pulp import *
import matplotlib.pyplot as plt

#Formatting
with open('bakery.txt') as f:
    lines = f.readlines()
pastries=[]
for line in lines:
    pastries+=[[int(line.split()[i]) for i in range(4)]]

ready=[pastries[i][1] for i in range(17)]
deadline=[pastries[i][2] for i in range(17)]
baking=[pastries[i][3] for i in range(17)]


prob = LpProblem("myProblem", LpMinimize)

#variables for starting times
starting_t=list(range(17))
start_vars = LpVariable.dicts("s",starting_t,lowBound=0,upBound=max(deadline))

#Extra variable
a = LpVariable("a",lowBound=0,upBound=max(deadline))

#The objective
prob+=a

#constraints
for i in range(17):
    #constraints to express the minimax problem
    prob+= a >= start_vars[i] + baking[i]
    #deadline constraints
    prob+= deadline[i] >= start_vars[i] + baking[i]
    #readiness constraints
    prob+= ready[i] <= start_vars[i]

#M is the maximum of deadlines
M=27000

#binary variables. 17 choose 2 is 136
binary_vars_indices=list(range(136))
binary_vars = LpVariable.dicts("z",binary_vars_indices,cat='Binary')

#no oven overlap constrains
count=0
for i in range(1,17):
    for j in range(i):
        M_i=deadline[i]
        M_j=deadline[j]
        prob+= start_vars[i] + baking[i] <= start_vars[j] + M_i*binary_vars[count]
        prob+= start_vars[j] + baking[j] <= start_vars[i] + M_j*(1-binary_vars[count])
        count+=1

prob.solve(PULP_CBC_CMD(msg=1))
#print(LpStatus[prob.status])



start_vars_list=prob.variables()[1:18]

s1=sorted(start_vars_list,key=lambda x:int(x.name[2:]))
for v in s1:
    print("{:<5s}{:<9.1f}".format(v.name.replace('_', '')+":",v.varValue))

#---------------------Visualize---------------------#

r2=[]
s2=[]
e2=[]
d2=[]
i2=[]
s=sorted(start_vars_list,key=lambda x:x.varValue)
for v in s:
    index=int(v.name[2:])    
    var_name=v.name.replace('_', '')+":"    
    ready_time=ready[index]
    start_time=v.varValue
    deadline_time=deadline[index]
    end_time=v.varValue + baking[index]

    i2+=[index]
    r2+=[ready_time]
    s2+=[start_time]
    e2+=[end_time]
    d2+=[deadline_time]
    #print("{:<5s}{:<9.1f}".format(var_name,start_time))
    #print("{:<5s}{:<9.1f}{:<9.1f}{:>9.1f}{:>9.1f}".format(var_name,ready_time,start_time,end_time,deadline_time))
    
fig = plt.figure()  
fig, ax = plt.subplots()
ax.grid(color="0.95")
ax.set_axisbelow(True)
#Empty plots for the legend
ax.plot([],"green",label="Timeframe Available to Bake",alpha=0.3)
ax.broken_barh([(0,0)],(0,0),label="Time in the Oven",facecolors =('purple'),alpha=0.2)
ax.scatter([e2[0]],[17],alpha=1,label="Time Indicator",color="black")
ax.scatter([e2[0]],[17],alpha=1,label="Critical Preparations",color="blue",marker="*")
ax.plot([],"red",label="Pastery ID Label")

for i in range(17):    
    ax.plot([r2[i],d2[i]],[17-i,17-i],"green",marker="|",alpha=0.3)
    ax.broken_barh([(s2[i],e2[i]-s2[i])], (16.5-i,1), facecolors =('purple'),alpha=0.2)
    
    label= "{}".format(int(e2[i]),int(e2[i]))
    ax.scatter([e2[i]],[17-i],color="black",marker="o")
    ax.text(e2[i]+50,17.2-i,label)
    ax.text((s2[i]+e2[i])/2,17.1-i,i2[i],ha="center",color="red",weight="bold")
    #Critical Preparations
    if r2[i]==s2[i]:
        ax.scatter([r2[i]],[17-i],color="blue",marker="*")

ax.set_xlabel("Time in seconds")
ax.set_ylabel("Pastery ID")
ax.set_title("Pastry Baking Schedule")
ax.set_yticks(list(range(1,18)),list(reversed(i2)))


ax.legend()
fig.show()





